In [2]:
import keras

In [3]:
import tensorflow as tf

In [4]:
model=tf.keras.models.load_model('Brain1.keras')

In [5]:
import cv2
import mediapipe as mp
import numpy as np
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
import matplotlib.pyplot as plt

In [6]:
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

# Function to calculate the angle between three points
def calculate_angle(a, b, c):
    a = np.array(a)  # First point
    b = np.array(b)  # Middle point (the joint)
    c = np.array(c)  # Last point

    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)

    if angle > 180.0:
        angle = 360.0 - angle

    return angle

cap = cv2.VideoCapture("C:\\Users\\Poonam\\OneDrive - Ishaan\\Desktop\\Ishaan\\Assignments\\Sem5\\FML\\Project\\Codes_FML\\archive\\Correct sequence\\Copy of push up 1.mp4")

# Push-up counter variables
counter = 0
stage = None

# Setup Mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()

        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Make detection
        results = pose.process(image)

        # Recolor back to BGR for rendering
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        Y_pred = model.predict(image)
        Y_pred = Y_pred>0.5
        if (Y_pred==0):
            pred = 'Correct'
        else:
            pred = 'Wrong'

            print("Our Prediction is :", pred)

        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark

            # Get coordinates for angle 1 (shoulder-hip-knee)
            shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]

            # Calculate angle 1 (shoulder-hip-knee)
            angle_1 = calculate_angle(shoulder, hip, knee)

            # Get coordinates for angle 2 (shoulder-elbow-wrist)
            elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]

            # Calculate angle 2 (shoulder-elbow-wrist)
            angle_2 = calculate_angle(shoulder, elbow, wrist)

          

            # Visualize both angles on the video feed
            cv2.putText(image, f"Angle 1: {int(angle_1)}",
                        tuple(np.multiply(hip, [640, 480]).astype(int)),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)

            cv2.putText(image, f"Angle 2: {int(angle_2)}",
                        tuple(np.multiply(elbow, [640, 480]).astype(int)),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)

            # Push-up counter logic
            if angle_1 > 160 and angle_2 > 160:  # Body is up
                stage = "up"
            if angle_1 > 160 and angle_2 < 40 and stage == "up":  # Body is down
                stage = "down"
                counter += 1
                print(counter)

        except:
            pass

        # Render push-up counter
        # Setup status box
        cv2.rectangle(image, (0, 0), (225, 73), (0, 165, 255), -1)

        # Rep data
        cv2.putText(image, "REPS:", (15, 12),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
        cv2.putText(image, str(counter),
                    (10, 60),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

        # Render pose landmarks
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                                  mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2))

        # Display video feed with landmarks
        cv2.imshow('Push-up Counter', image)

        # Break loop on 'q' key press
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

   

# Release camera and close windows
cap.release()
cv2.destroyAllWindows()

ValueError: Exception encountered when calling MaxPooling2D.call().

[1mNegative dimension size caused by subtracting 2 from 1 for '{{node sequential_1/max_pooling2d_1/MaxPool2d}} = MaxPool[T=DT_FLOAT, data_format="NHWC", explicit_paddings=[], ksize=[1, 2, 2, 1], padding="VALID", strides=[1, 2, 2, 1]](sequential_1/conv2d_1/Relu)' with input shapes: [32,638,1,32].[0m

Arguments received by MaxPooling2D.call():
  • inputs=tf.Tensor(shape=(32, 638, 1, 32), dtype=float32)